In [1]:
import math
import random
import pandas as pd
import numpy as np

In [2]:
# Funcao Objetivo
def f_alpine02(x):
    resultado = 1
    for i, x_i in x.iteritems():
        resultado = resultado * math.sin(x_i) * math.sqrt(x_i)
    return resultado

In [3]:
# Ordenar
def ordena(df):
    df['aux'] = df.apply(f_alpine02, axis=1)
    df = df.sort_values('aux', ascending=False).drop('aux', axis = 1)
    return df.reset_index(drop=True)

In [4]:
def ranking_linear(df, max, min):
    Fx = []
    df_ordenado = ordena(df)
    df_ordenado = df_ordenado.reset_index(drop = True)
    for i, row in df_ordenado.iterrows():
        resultado = min + (max - min)*(( len(df_ordenado) - i - 1)/( len(df_ordenado) -1))
        Fx.append(resultado)
    return Fx

In [5]:
# Mutacao
def mutacao_uniforme(individuo, tx_mutacao):
    individuo_mutado = []
    # como a taxa de mutacao eh entre 0 e 1, temos:
    for x_i in individuo:
        rand = np.random.random()
        if rand < tx_mutacao:
            x_i = np.random.uniform(0.0, 10.0)
        individuo_mutado.append(x_i)
    return individuo_mutado

In [6]:
def crossover(P1, P2):
    C1 = P1
    C2 = P2
    alpha = np.random.random()
    #print('olha ele aqui ----', alpha)
    for i in range(len(P1)):
        t = alpha*P1[i] + (1 - alpha)*P2[i]
        s = alpha*P2[i] + (1 - alpha)*P1[i]

        if t>=0 and t<=10 and s<=10 and s>=0:
            C1[i] = t
            C2[i] = s
    return (C1, C2)

In [7]:
# Netodo de selecao - roleta
def selecao_roleta(Fx, df):
    posicao = 0
    soma_acumulada = np.cumsum(Fx)
    tamanho = len(Fx)
    limite = soma_acumulada[tamanho-1]
    rand = np.random.uniform(0.0, limite)

    for i, valor in enumerate(soma_acumulada):
        if rand <= valor:
            posicao = i
            break
    return df.loc[posicao]

In [8]:
def selecao_torneio(df,n):
    vencendor = None
    for i in range(n):
        i = np.random.randint(0,len(df))
        if(vencendor == None):
            vencedor = df.loc[i]
        elif (vencedor < f_alpine02(df.loc[i])):
            vencendor = df.loc[i]
    
    return vencedor
    
    

In [9]:
from tqdm import tqdm
# define tamanho da populacao
tamanho_pop: int = 100
tx_mutacao = 0.05

# Cria dataframe e preenche com valores randomicos
populacao = pd.DataFrame(np.random.uniform(0.0, 10.0, size=(tamanho_pop, 2)), columns=['X1', 'X2'])

#print(populacao)

# melhor
best = []
pior = []
melhor = []
media = []
# num geracoes
total_geracoes = 50

for geracao in tqdm(range(total_geracoes)) : 
    # ordena
    populacao_ordenada = ordena(populacao.copy()).reset_index(drop = True)
    # Pegar melhor individuo da geracao atual
    if geracao == 0:
        best = populacao_ordenada.loc[0]
    elif f_alpine02(best) < f_alpine02(populacao_ordenada.loc[0]):
        best = populacao_ordenada.loc[0]
        
    melhor.append(populacao_ordenada.loc[0])
    pior.append(populacao_ordenada.loc[99])
    #media.append()

    Fx = ranking_linear(populacao_ordenada, 10, 0)
    populacao_ordenada = populacao_ordenada.drop('aux', axis=1)

    nova_populacao = pd.DataFrame(columns = ['X1', 'X2'])
    for i in range(0, int(tamanho_pop/2)):
        # chama selecao
        P1 = selecao_roleta(Fx, populacao_ordenada)
        P2 = selecao_roleta(Fx, populacao_ordenada)

        # transforma P1 e P2 em vetor
        P1 = P1.to_numpy()
        P2 = P2.to_numpy()

        # faz crossover
        C1, C2 = crossover(P1, P2)

        # Realizar mutacao de C1 e C2
        C1 = mutacao_uniforme(C1, tx_mutacao)
        C2 = mutacao_uniforme(C2, tx_mutacao)
        
        nova_populacao=nova_populacao.append(pd.DataFrame(data=[C1, C2], columns = ['X1', 'X2'] ))
 
    populacao = nova_populacao.reset_index(drop=True)

print(f_alpine02(best))
'''for i in melhor:
    print(f_alpine02(i))
print()
for i in pior:
    print(f_alpine02(i))'''

100%|██████████| 50/50 [00:06<00:00,  7.96it/s]

7.673267134254619


'for i in melhor:\n    print(f_alpine02(i))\nprint()\nfor i in pior:\n    print(f_alpine02(i))'